# Notebook for plots

In [1]:
using JupyterFormatter
enable_autoformat()

1-element Vector{Function}:
 format_current_cell (generic function with 1 method)

In [2]:
using HalfIntegers
using CSV
using DataFrames
using Statistics

# set Plot style
using Plots
using Plots.PlotMeasures
using LaTeXStrings

# set backend and style
pgfplotsx()
theme(:default)
default(
    markerstrokewidth = 0,
    markerstrokealpha = 0,
    linewidth = 2,
    grid = :true,
    gridlinewidth = 0.5,
    markersize = 5,
    tickfontsize = 22,
    size = (900, 600),
    legendfontsize = 22,
    guidefontsize = 22,
    titlefontsize = 22,
    legend = (0.03, 0.98),
    foreground_color_axis = "black",
    foreground_color_border = "black",
    foreground_color_guide = "darkorange",
    foreground_color_text = "black",
    guidefontcolor = "black",
    plot_titlefontcolor = "black",
    titlefontcolor = "black",
    shape = [:circle],
    markershape = [:circle],
)

In [3]:
FOLDER_WITH_DATA = "../../"

"../../"

# Self energy

## BF self-energy

In [ ]:
CUTOFF = 10
JB = 0.5
IB = 0
TRIALS_TO_AVERAGE = 20;

In [37]:
# vector with amplitudes computed exactly
ampls = vec(
    Matrix(
        DataFrame(
            CSV.File(
                "$(FOLDER_WITH_DATA)/data/self_energy/jb_$(JB)/exact/BF/ampls_cutoff_$(CUTOFF)_ib_$(IB).0.csv",
            ),
        ),
    ),
)

for MC_ITERATIONS in [10^3, 10^4]

    AMPLS_MATRIX = zeros(TRIALS_TO_AVERAGE, 2 * CUTOFF + 1)

    for trial = 1:TRIALS_TO_AVERAGE

        ampls_MC = Matrix(
            DataFrame(
                CSV.File(
                    "$(FOLDER_WITH_DATA)/data/self_energy/jb_$(JB)/monte_carlo/Nmc_$(MC_ITERATIONS)/BF/ampls_cutoff_$(CUTOFF)_ib_$(IB).0_trial_$(trial).csv",
                ),
            ),
        )

        AMPLS_MATRIX[trial, :] .= ampls_MC[:, 1]

    end

    AMPLS_MEAN_VEC = mean(AMPLS_MATRIX, dims = 1)
    AMPLS_STD_VEC = std(AMPLS_MATRIX, dims = 1)

    pl1 = plot(
        0.5:0.5:10,
        [ampls[2:end], AMPLS_MEAN_VEC[2:end]],
        label = ["Exact amplitude" "MC estimate"],
        markershape = [:circle :circle :circle],
        shape = [:circle :circle :circle],
        xticks = 1:1:10,
        legend = (0.10, 0.97),
        yscale = :log10,
        xscale = :log10,
        palette = palette([:blue, :skyblue], 2),
        legendfontsize = 22,
    )
    xlabel!(L"K")
    title!(L"\textrm{Self energy BF} \left(j_b=%$JB, \, i_b=%$IB \right)")
    savefig(
        "./plots/BF/self_energy/MC_$(MC_ITERATIONS)_trials_$(TRIALS_TO_AVERAGE)_with_exact.pdf",
    )

    pl1 = plot(
        0.5:0.5:10,
        [AMPLS_MEAN_VEC[2:end]],
        label = ["MC estimate"],
        markershape = [:circle :circle :circle],
        shape = [:circle :circle :circle],
        yerror = AMPLS_STD_VEC[2:end],
        xticks = 1:1:10,
        legend = (0.10, 0.97),
        yscale = :log10,
        xscale = :log10,
        markerstrokewidth = 10,
        #palette = palette([:skyblue, :red], 2),
        legendfontsize = 22,
    )
    xlabel!(L"K")
    title!(L"\textrm{Self energy BF} \left(j_b=%$JB, \, i_b=%$IB \right)")
    savefig(
        "./plots/BF/self_energy/MC_$(MC_ITERATIONS)_trials_$(TRIALS_TO_AVERAGE)_with_error.pdf",
    )


end

## EPRL self-energy

In [22]:
IMMIRZI = 0.1
DL_MAX = 10;

We first plot each $\Delta l$ separately

In [105]:
for Dl = 0:DL_MAX

    mkpath("./plots/EPRL/self_energy/immirzi_$(IMMIRZI)/Dl_$(Dl)")

    # vector with amplitudes computed exactly
    ampls = Matrix(
        DataFrame(
            CSV.File(
                "$(FOLDER_WITH_DATA)/data/self_energy/jb_$(JB)/exact/EPRL/immirzi_$(IMMIRZI)/Dl_$(Dl)/ampls_cutoff_$(CUTOFF)_ib_$(IB).0.csv",
            ),
        ),
    )

    for MC_ITERATIONS in [10^3, 10^4, 10^5]

        AMPLS_MATRIX = zeros(TRIALS_TO_AVERAGE, 2 * CUTOFF + 1)

        for trial = 1:TRIALS_TO_AVERAGE

            ampls_MC = Matrix(
                DataFrame(
                    CSV.File(
                        "$(FOLDER_WITH_DATA)/data/self_energy/jb_$(JB)/monte_carlo/Nmc_$(MC_ITERATIONS)/EPRL/immirzi_$(IMMIRZI)/Dl_$(Dl)/ampls_cutoff_$(CUTOFF)_ib_$(IB).0_trial_$(trial).csv",
                    ),
                ),
            )

            AMPLS_MATRIX[trial, :] .= ampls_MC[:, 1]

        end

        AMPLS_MEAN_VEC = mean(AMPLS_MATRIX, dims = 1)
        AMPLS_STD_VEC = std(AMPLS_MATRIX, dims = 1)

        pl1 = plot(
            0:0.5:10,
            [ampls[:], AMPLS_MEAN_VEC[:]],
            label = ["Exact amplitude" "MC estimate"],
            markershape = [:circle :circle],
            shape = [:circle :circle],
            xticks = 1:1:10,
            grid = :true,
            legend = (0.10, 0.97),
            palette = palette([:blue, :skyblue], 2),
            legendfontsize = 22,
        )
        xlabel!(L"K")
        title!(L"\textrm{Self energy EPRL} \left(j_b=%$JB, \, i_b=%$IB \right)")
        savefig(
            "./plots/EPRL/self_energy/immirzi_$(IMMIRZI)/Dl_$(Dl)/MC_$(MC_ITERATIONS)_trials_$(TRIALS_TO_AVERAGE)_with_exact.pdf",
        )


        pl1 = plot(
            0:0.5:10,
            [AMPLS_MEAN_VEC[:]],
            label = ["MC estimate"],
            markershape = [:circle :circle],
            shape = [:circle :circle],
            yerror = AMPLS_STD_VEC[:],
            xticks = 1:1:10,
            grid = :true,
            legend = (0.10, 0.97),
            markerstrokewidth = 10,
            #palette = palette([:skyblue, :red], 2),
            legendfontsize = 22,
        )
        xlabel!(L"K")
        title!(L"\textrm{Self energy EPRL} \left(j_b=%$JB, \, i_b=%$IB \right)")
        savefig(
            "./plots/EPRL/self_energy/immirzi_$(IMMIRZI)/Dl_$(Dl)/MC_$(MC_ITERATIONS)_trials_$(TRIALS_TO_AVERAGE)_with_error.pdf",
        )

    end

end

### Extrapolation

In [100]:
AMPLS_EXACT_ALL_Dl = zeros(2 * CUTOFF + 1, DL_MAX + 1)

vec_MC = [10^3, 10^4, 10^5]
number_of_MC_param = size(vec_MC)[1]

AMPLS_MC_ALL_Dl = zeros(2 * CUTOFF + 1, DL_MAX + 1, TRIALS_TO_AVERAGE, number_of_MC_param)

mkpath("./plots/EPRL/self_energy/immirzi_$(IMMIRZI)/extrapolation")

#= PART 1
   We first fill the tensors with the amplitudes (exact and monte carlo) computed numerically
=#

for Dl = 0:DL_MAX

    # vector with amplitudes computed exactly
    ampls = Matrix(
        DataFrame(
            CSV.File(
                "$(FOLDER_WITH_DATA)/data/self_energy/jb_$(JB)/exact/EPRL/immirzi_$(IMMIRZI)/Dl_$(Dl)/ampls_cutoff_$(CUTOFF)_ib_$(IB).0.csv",
            ),
        ),
    )

    AMPLS_EXACT_ALL_Dl[:, Dl+1] .= ampls[:]

    for MC_ITERATIONS_index in eachindex(vec_MC)

        for trial = 1:TRIALS_TO_AVERAGE

            ampls_MC = Matrix(
                DataFrame(
                    CSV.File(
                        "$(FOLDER_WITH_DATA)/data/self_energy/jb_$(JB)/monte_carlo/Nmc_$(vec_MC[MC_ITERATIONS_index])/EPRL/immirzi_$(IMMIRZI)/Dl_$(Dl)/ampls_cutoff_$(CUTOFF)_ib_$(IB).0_trial_$(trial).csv",
                    ),
                ),
            )

            AMPLS_MC_ALL_Dl[:, Dl+1, trial, MC_ITERATIONS_index] .= ampls_MC[:, 1]

        end

    end

end


#= PART 2
   Then we extrapolate the amplitudes computed exactly and with monte carlo
=#


AMPLS_EXACT_EXTRAPOLATED = zeros(2 * CUTOFF + 1)

AMPLS_MC_EXTRAPOLATED_INTERM = zeros(2 * CUTOFF + 1, TRIALS_TO_AVERAGE, number_of_MC_param)


for K_index = 2:(2*CUTOFF+1)

    for Dl = 2:DL_MAX

        Dl_index = Dl + 1

        # extrapolating exact amplitudes

        AMPLS_EXACT_EXTRAPOLATED[K_index] =
            (
                AMPLS_EXACT_ALL_Dl[K_index, Dl_index] *
                AMPLS_EXACT_ALL_Dl[K_index, Dl_index-2] -
                (AMPLS_EXACT_ALL_Dl[K_index, Dl_index-1])^2
            ) / (
                AMPLS_EXACT_ALL_Dl[K_index, Dl_index] -
                2 * AMPLS_EXACT_ALL_Dl[K_index, Dl_index-1] +
                AMPLS_EXACT_ALL_Dl[K_index, Dl_index-2]
            )

        for MC_ITERATIONS_index in eachindex(vec_MC)

            for trial = 1:TRIALS_TO_AVERAGE

                # extrapolating monte carlo amplitudes

                AMPLS_MC_EXTRAPOLATED_INTERM[K_index, trial, MC_ITERATIONS_index] =
                    (
                        AMPLS_MC_ALL_Dl[K_index, Dl_index, trial, MC_ITERATIONS_index] *
                        AMPLS_MC_ALL_Dl[K_index, Dl_index-2, trial, MC_ITERATIONS_index] -
                        (AMPLS_MC_ALL_Dl[
                            K_index,
                            Dl_index-1,
                            trial,
                            MC_ITERATIONS_index,
                        ])^2
                    ) / (
                        AMPLS_MC_ALL_Dl[K_index, Dl_index, trial, MC_ITERATIONS_index] -
                        2 *
                        AMPLS_MC_ALL_Dl[K_index, Dl_index-1, trial, MC_ITERATIONS_index] +
                        AMPLS_MC_ALL_Dl[K_index, Dl_index-2, trial, MC_ITERATIONS_index]
                    )

            end

        end

    end

end


#= PART 3
   Finally, compute average and standard dev over the trials for Monte Carlo amplitudes
=#

AMPLS_MC_EXTRAPOLATED = zeros(2 * CUTOFF + 1, number_of_MC_param, 2)

for K_index = 1:(2*CUTOFF+1)

    for MC_ITERATIONS_index in eachindex(vec_MC)

        AMPLS_MC_EXTRAPOLATED[K_index, MC_ITERATIONS_index, 1] =
            mean(AMPLS_MC_EXTRAPOLATED_INTERM[K_index, :, MC_ITERATIONS_index])
        AMPLS_MC_EXTRAPOLATED[K_index, MC_ITERATIONS_index, 2] =
            std(AMPLS_MC_EXTRAPOLATED_INTERM[K_index, :, MC_ITERATIONS_index])

    end

end

We now plot the extrapolated amplitudes

In [108]:
for MC_ITERATIONS_index in eachindex(vec_MC)

    pl1 = plot(
        0:0.5:10,
        [AMPLS_EXACT_EXTRAPOLATED[:], AMPLS_MC_EXTRAPOLATED[:, MC_ITERATIONS_index, 1]],
        label = ["Exact amplitude" "MC estimate"],
        markershape = [:circle :circle],
        shape = [:circle :circle],
        xticks = 1:1:10,
        grid = :true,
        legend = (0.10, 0.97),
        palette = palette([:blue, :skyblue], 2),
        legendfontsize = 22,
    )
    xlabel!(L"K")
    title!(L"\textrm{Self energy EPRL} \left(j_b=%$JB, \, i_b=%$IB \right)")
    savefig(
        "./plots/EPRL/self_energy/immirzi_$(IMMIRZI)/extrapolation/MC_$(vec_MC[MC_ITERATIONS_index])_trials_$(TRIALS_TO_AVERAGE)_Dl_max_$(DL_MAX)_with_exact.pdf",
    )

    pl1 = plot(
        0:0.5:10,
        [AMPLS_MC_EXTRAPOLATED[:, MC_ITERATIONS_index, 1]],
        label = ["MC estimate"],
        markershape = [:circle :circle],
        shape = [:circle :circle],
        yerror = AMPLS_MC_EXTRAPOLATED[:, MC_ITERATIONS_index, 2],
        xticks = 1:1:10,
        grid = :true,
        legend = (0.10, 0.97),
        markerstrokewidth = 10,
        #palette = palette([:skyblue, :red], 2),
        legendfontsize = 22,
    )
    xlabel!(L"K")
    title!(L"\textrm{Self energy EPRL} \left(j_b=%$JB, \, i_b=%$IB \right)")
    savefig(
        "./plots/EPRL/self_energy/immirzi_$(IMMIRZI)/extrapolation/MC_$(vec_MC[MC_ITERATIONS_index])_trials_$(TRIALS_TO_AVERAGE)_Dl_max_$(DL_MAX)_with_error.pdf",
    )

end

## BF vertex renormalization

In [6]:
CUTOFF = 10
JB = 0.5
IB = 0
TRIALS_TO_AVERAGE = 4;

In [20]:
# vector with the amplitudes computed exactly
if (CUTOFF <= 4)
    ampls = vec(
        Matrix(
            DataFrame(
                CSV.File(
                    "$(FOLDER_WITH_DATA)/data/vertex_renormalization/jb_$(JB)/exact/BF/ampls_cutoff_$(CUTOFF).0_ib_$(IB).0.csv",
                ),
            ),
        ),
    )
end

for MC_ITERATIONS in [10^5]

    AMPLS_MATRIX = zeros(TRIALS_TO_AVERAGE, 2 * CUTOFF + 1)

    for trial = 1:TRIALS_TO_AVERAGE

        ampls_MC = Matrix(
            DataFrame(
                CSV.File(
                    "$(FOLDER_WITH_DATA)/data/vertex_renormalization/jb_$(JB)/monte_carlo/Nmc_$(MC_ITERATIONS)/BF/ampls_cutoff_$(CUTOFF)_ib_$(IB).0_trial_$(trial).csv",
                ),
            ),
        )

        AMPLS_MATRIX[trial, :] .= ampls_MC[:, 1]

    end

    AMPLS_MEAN_VEC = mean(AMPLS_MATRIX, dims = 1)
    AMPLS_STD_VEC = std(AMPLS_MATRIX, dims = 1)

    if (CUTOFF <= 4)
        pl1 = plot(
            0.5:0.5:CUTOFF,
            [ampls[2:end], AMPLS_MEAN_VEC[2:end]],
            label = ["Exact amplitude" "MC estimate"],
            markershape = [:circle :circle :circle],
            shape = [:circle :circle :circle],
            xticks = 1:1:10,
            legend = (0.10, 0.97),
            yscale = :log10,
            xscale = :log10,
            palette = palette([:blue, :skyblue], 2),
            legendfontsize = 22,
        )
        xlabel!(L"K")
        title!(L"\textrm{Vertex renormalization BF} \left(j_b=%$JB, \, i_b=%$IB \right)")
        savefig(
            "./plots/BF/vertex_renormalization_MC_$(MC_ITERATIONS)_plus_exact_trials_$(TRIALS_TO_AVERAGE).pdf",
        )
    end

    pl1 = plot(
        0.5:0.5:CUTOFF,
        [AMPLS_MEAN_VEC[2:end]],
        label = ["MC estimate"],
        markershape = [:circle :circle :circle],
        shape = [:circle :circle :circle],
        yerror = AMPLS_STD_VEC[2:end],
        xticks = 1:1:10,
        legend = (0.10, 0.97),
        yscale = :log10,
        xscale = :log10,
        markerstrokewidth = 10,
        #palette = palette([:skyblue, :red], 2),
        legendfontsize = 22,
    )
    xlabel!(L"K")
    title!(L"\textrm{Vertex renormalization BF} \left(j_b=%$JB, \, i_b=%$IB \right)")
    savefig(
        "./plots/BF/vertex_renormalization_MC_$(MC_ITERATIONS)_trials_$(TRIALS_TO_AVERAGE).pdf",
    )

    check = Float64[]

for i in 0:0.5:10
  push!(check, (3/4)^4*i^12)  
end

pl1 = plot(0.5:0.5:CUTOFF, 
    [AMPLS_MEAN_VEC[2:end], check[2:end]],
    label=["computed" L"(0.75)^4 \cdot K^{12}"],
    markershape = [:circle :star],
    shape = [:circle :star],
    xticks = 1:1:10,
    xscale=:log10,
    yscale=:log10,
    legend=(0.10,0.97), 
    palette = palette([:skyblue, :royalblue2]),
    legendfontsize = 22)
    xlabel!(L"K")
    title!(L"\textrm{Vert. renorm. BF scaling check} \, \left(j_b=0.5, \, i_b=0 \right)")
savefig("./plots/BF/vertex_renorm_check.pdf")  


end

## EPRL vertex renormalization

We first plot each $\Delta l$ separately

We need to normalize the amplitudes because the julia plots is unable to handle very small numbers

In [12]:
NORMALIZATION_PLOTS = 1000000000000000;

In [13]:
TRIALS_TO_AVERAGE = 20
IMMIRZI = 0.1
DL_MAX = 10;

In [15]:
for Dl = 0:DL_MAX

    mkpath("./plots/EPRL/vertex_renormalization/immirzi_$(IMMIRZI)/Dl_$(Dl)")

    for MC_ITERATIONS in [10^5]

        AMPLS_MATRIX = zeros(TRIALS_TO_AVERAGE, 2 * CUTOFF + 1)

        for trial = 1:TRIALS_TO_AVERAGE

            ampls_MC = Matrix(
                DataFrame(
                    CSV.File(
                        "$(FOLDER_WITH_DATA)/data/vertex_renormalization/jb_$(JB)/monte_carlo/Nmc_$(MC_ITERATIONS)/EPRL/immirzi_$(IMMIRZI)/Dl_$(Dl)/ampls_cutoff_$(CUTOFF)_ib_$(IB).0_trial_$(trial).csv",
                    ),
                ),
            )

            ampls_MC[:] .= ampls_MC[:] .* NORMALIZATION_PLOTS

            AMPLS_MATRIX[trial, :] .= ampls_MC[:, 1]

        end

        AMPLS_MEAN_VEC = mean(AMPLS_MATRIX, dims = 1)
        AMPLS_STD_VEC = std(AMPLS_MATRIX, dims = 1)

        pl1 = plot(
            0:0.5:CUTOFF,
            [AMPLS_MEAN_VEC[:]],
            label = "",
            markershape = [:circle :circle],
            shape = [:circle :circle],
            yerror = AMPLS_STD_VEC[:],
            xticks = 1:1:10,
            grid = :true,
            legend = (0.10, 0.97),
            markerstrokewidth = 10,
            #palette = palette([:skyblue, :red], 2),
            legendfontsize = 22,
        )
        xlabel!(L"K")
        title!(L"\textrm{Vertex renormalization EPRL} \left(j_b=%$JB, \, i_b=%$IB \right)")
        savefig(
            "./plots/EPRL/vertex_renormalization/immirzi_$(IMMIRZI)/Dl_$(Dl)/MC_$(MC_ITERATIONS)_trials_$(TRIALS_TO_AVERAGE)_with_error.pdf",
        )

    end

end

### Extrapolation

In [19]:
vec_MC = [10^5]
number_of_MC_param = size(vec_MC)[1]

AMPLS_MC_ALL_Dl = zeros(2 * CUTOFF + 1, DL_MAX + 1, TRIALS_TO_AVERAGE, number_of_MC_param)

mkpath("./plots/EPRL/vertex_renormalization/immirzi_$(IMMIRZI)/extrapolation")

#= PART 1
   We first fill the tensors with the monte carlo amplitudes computed numerically
=#

for Dl = 0:DL_MAX

    for MC_ITERATIONS_index in eachindex(vec_MC)

        for trial = 1:TRIALS_TO_AVERAGE

            ampls_MC = Matrix(
                DataFrame(
                    CSV.File(
                        "$(FOLDER_WITH_DATA)/data/vertex_renormalization/jb_$(JB)/monte_carlo/Nmc_$(vec_MC[MC_ITERATIONS_index])/EPRL/immirzi_$(IMMIRZI)/Dl_$(Dl)/ampls_cutoff_$(CUTOFF)_ib_$(IB).0_trial_$(trial).csv",
                    ),
                ),
            )

            ampls_MC[:] .= ampls_MC[:] .* NORMALIZATION_PLOTS

            AMPLS_MC_ALL_Dl[:, Dl+1, trial, MC_ITERATIONS_index] .= ampls_MC[:, 1]

        end

    end

end


#= PART 2
   Then we extrapolate the amplitudes computed with monte carlo
=#


AMPLS_MC_EXTRAPOLATED_INTERM = zeros(2 * CUTOFF + 1, TRIALS_TO_AVERAGE, number_of_MC_param)


for K_index = 2:(2*CUTOFF+1)

    for Dl = 2:DL_MAX

        Dl_index = Dl + 1

        for MC_ITERATIONS_index in eachindex(vec_MC)

            for trial = 1:TRIALS_TO_AVERAGE

                # extrapolating monte carlo amplitudes

                AMPLS_MC_EXTRAPOLATED_INTERM[K_index, trial, MC_ITERATIONS_index] =
                    (
                        AMPLS_MC_ALL_Dl[K_index, Dl_index, trial, MC_ITERATIONS_index] *
                        AMPLS_MC_ALL_Dl[K_index, Dl_index-2, trial, MC_ITERATIONS_index] -
                        (AMPLS_MC_ALL_Dl[
                            K_index,
                            Dl_index-1,
                            trial,
                            MC_ITERATIONS_index,
                        ])^2
                    ) / (
                        AMPLS_MC_ALL_Dl[K_index, Dl_index, trial, MC_ITERATIONS_index] -
                        2 *
                        AMPLS_MC_ALL_Dl[K_index, Dl_index-1, trial, MC_ITERATIONS_index] +
                        AMPLS_MC_ALL_Dl[K_index, Dl_index-2, trial, MC_ITERATIONS_index]
                    )

            end

        end

    end

end


#= PART 3
   Finally, compute average and standard dev over the trials for Monte Carlo amplitudes
=#

AMPLS_MC_EXTRAPOLATED = zeros(2 * CUTOFF + 1, number_of_MC_param, 2)

for K_index = 1:(2*CUTOFF+1)

    for MC_ITERATIONS_index in eachindex(vec_MC)

        AMPLS_MC_EXTRAPOLATED[K_index, MC_ITERATIONS_index, 1] =
            mean(AMPLS_MC_EXTRAPOLATED_INTERM[K_index, :, MC_ITERATIONS_index])
        AMPLS_MC_EXTRAPOLATED[K_index, MC_ITERATIONS_index, 2] =
            std(AMPLS_MC_EXTRAPOLATED_INTERM[K_index, :, MC_ITERATIONS_index])

    end

end

We now plot the extrapolated amplitudes

In [21]:
for MC_ITERATIONS_index in eachindex(vec_MC)

    pl1 = plot(
        0:0.5:CUTOFF,
        [AMPLS_MC_EXTRAPOLATED[:, MC_ITERATIONS_index, 1]],
        label = "",
        markershape = [:circle :circle],
        shape = [:circle :circle],
        yerror = AMPLS_MC_EXTRAPOLATED[:, MC_ITERATIONS_index, 2],
        xticks = 1:1:10,
        grid = :true,
        legend = (0.10, 0.97),
        markerstrokewidth = 10,
        #palette = palette([:skyblue, :red], 2),
        legendfontsize = 22,
    )
    xlabel!(L"K")
    title!(L"\textrm{Vertex renormalization EPRL} \left(j_b=%$JB, \, i_b=%$IB \right)")
    savefig(
        "./plots/EPRL/vertex_renormalization/immirzi_$(IMMIRZI)/extrapolation/MC_$(vec_MC[MC_ITERATIONS_index])_trials_$(TRIALS_TO_AVERAGE)_Dl_max_$(DL_MAX)_with_error.pdf",
    )

end